<a href="https://colab.research.google.com/github/sidsanc/258_DeepLearningAssignment/blob/main/Assignment6/DL_Pipeline_custom_components.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt


Load the MNIST dataset:

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255

11490434/11490434 [==============================] - 0s 0us/step


#Custom Loss Function

In [4]:
# Custom loss function
def huber_loss(y_true, y_pred):
  y_true = tf.cast(y_true, tf.float32)
  error = y_true - y_pred
  is_small_error = tf.abs(error) < 1
  squared_loss = tf.square(error) / 2
  linear_loss  = tf.abs(error) - 0.5
  return tf.where(is_small_error, squared_loss, linear_loss)

# Model with custom loss function
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=huber_loss,
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_split=0.1)


Epoch 1/5
1688/1688 [==============================] - 76s 43ms/step - loss: 3.9111 - accuracy: 0.0909 - val_loss: 3.9427 - val_accuracy: 0.0650
Epoch 2/5
1688/1688 [==============================] - 72s 43ms/step - loss: 3.9111 - accuracy: 0.0930 - val_loss: 3.9427 - val_accuracy: 0.0857
Epoch 3/5
1688/1688 [==============================] - 67s 39ms/step - loss: 3.9111 - accuracy: 0.0969 - val_loss: 3.9427 - val_accuracy: 0.1268
Epoch 4/5
1688/1688 [==============================] - 65s 39ms/step - loss: 3.9111 - accuracy: 0.0981 - val_loss: 3.9427 - val_accuracy: 0.0978
Epoch 5/5
1688/1688 [==============================] - 72s 42ms/step - loss: 3.9111 - accuracy: 0.0961 - val_loss: 3.9427 - val_accuracy: 0.1082


Custom Regularizer

In [5]:
# Custom regularizer
class CustomRegularizer(tf.keras.regularizers.Regularizer):
  def __init__(self, factor):
      self.factor = factor

  def __call__(self, weights):
      return tf.reduce_sum(tf.abs(self.factor * weights))

model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=CustomRegularizer(0.001)),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_split=0.1)


Epoch 1/5
1688/1688 [==============================] - 66s 38ms/step - loss: 0.6964 - accuracy: 0.9248 - val_loss: 0.2825 - val_accuracy: 0.9777
Epoch 2/5
1688/1688 [==============================] - 65s 39ms/step - loss: 0.3250 - accuracy: 0.9622 - val_loss: 0.2552 - val_accuracy: 0.9805
Epoch 3/5
1688/1688 [==============================] - 66s 39ms/step - loss: 0.2850 - accuracy: 0.9674 - val_loss: 0.2352 - val_accuracy: 0.9818
Epoch 4/5
1688/1688 [==============================] - 62s 37ms/step - loss: 0.2665 - accuracy: 0.9694 - val_loss: 0.2400 - val_accuracy: 0.9757
Epoch 5/5
1688/1688 [==============================] - 62s 37ms/step - loss: 0.2557 - accuracy: 0.9706 - val_loss: 0.2286 - val_accuracy: 0.9790


Custom activation function

In [6]:
# Custom activation function
@tf.function
def custom_activation(x):
    return tf.maximum(0.0, x) * tf.sin(x)

# Model with custom activation function
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1)),
    layers.Activation(custom_activation),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3)),
    layers.Activation(custom_activation),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128),
    layers.Activation(custom_activation),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_split=0.1)


Epoch 1/5
1688/1688 [==============================] - 97s 57ms/step - loss: 0.1882 - accuracy: 0.9422 - val_loss: 0.0527 - val_accuracy: 0.9837
Epoch 2/5
1688/1688 [==============================] - 83s 49ms/step - loss: 0.0494 - accuracy: 0.9854 - val_loss: 0.0351 - val_accuracy: 0.9897
Epoch 3/5
1688/1688 [==============================] - 85s 50ms/step - loss: 0.0326 - accuracy: 0.9901 - val_loss: 0.0338 - val_accuracy: 0.9907
Epoch 4/5
1688/1688 [==============================] - 81s 48ms/step - loss: 0.0246 - accuracy: 0.9921 - val_loss: 0.0329 - val_accuracy: 0.9915
Epoch 5/5
1688/1688 [==============================] - 82s 49ms/step - loss: 0.0176 - accuracy: 0.9948 - val_loss: 0.0303 - val_accuracy: 0.9913


  Custom Initializer

In [7]:
def glorot_initializer(shape, dtype=tf.float32):
  stddev = tf.sqrt(2. / (shape[0] + shape[1]))
  return tf.random.normal(shape, stddev=stddev, dtype=dtype)

# Model with custom initializer
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1), kernel_initializer=glorot_initializer),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu', kernel_initializer=glorot_initializer),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_initializer=glorot_initializer),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_split=0.1)


Epoch 1/5
1688/1688 [==============================] - 65s 38ms/step - loss: 0.2984 - accuracy: 0.9164 - val_loss: 0.0750 - val_accuracy: 0.9775
Epoch 2/5
1688/1688 [==============================] - 64s 38ms/step - loss: 0.0948 - accuracy: 0.9717 - val_loss: 0.0574 - val_accuracy: 0.9850
Epoch 3/5
1688/1688 [==============================] - 63s 37ms/step - loss: 0.0647 - accuracy: 0.9806 - val_loss: 0.0617 - val_accuracy: 0.9832
Epoch 4/5
1688/1688 [==============================] - 63s 37ms/step - loss: 0.0472 - accuracy: 0.9854 - val_loss: 0.0391 - val_accuracy: 0.9887
Epoch 5/5
1688/1688 [==============================] - 63s 37ms/step - loss: 0.0373 - accuracy: 0.9882 - val_loss: 0.0467 - val_accuracy: 0.9885


Custom kernel constraint

In [8]:
def CustomConstraint(weights):
  return tf.where(weights < 0., tf.zeros_like(weights), weights)

# Model with custom kernel constraint
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1), kernel_constraint=CustomConstraint()),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu', kernel_constraint=CustomConstraint()),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_constraint=CustomConstraint()),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_split=0.1)


Epoch 1/5
1688/1688 [==============================] - 65s 38ms/step - loss: 0.1571 - accuracy: 0.9515 - val_loss: 0.0484 - val_accuracy: 0.9850
Epoch 2/5
1688/1688 [==============================] - 62s 37ms/step - loss: 0.0522 - accuracy: 0.9836 - val_loss: 0.0373 - val_accuracy: 0.9888
Epoch 3/5
1688/1688 [==============================] - 61s 36ms/step - loss: 0.0372 - accuracy: 0.9879 - val_loss: 0.0300 - val_accuracy: 0.9908
Epoch 4/5
1688/1688 [==============================] - 62s 37ms/step - loss: 0.0289 - accuracy: 0.9914 - val_loss: 0.0287 - val_accuracy: 0.9925
Epoch 5/5
1688/1688 [==============================] - 62s 37ms/step - loss: 0.0226 - accuracy: 0.9928 - val_loss: 0.0364 - val_accuracy: 0.9900


Custom Metrics

In [ ]:
class CustomAccuracy(tf.keras.metrics.Metric):
    def __init__(self, name="custom_accuracy", **kwargs):
        super(CustomAccuracy, self).__init__(name=name, **kwargs)
        self.correct = self.add_weight(name="correct", initializer="zeros")
        self.total = self.add_weight(name="total", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.argmax(y_true, axis=-1)
        y_pred = tf.argmax(y_pred, axis=-1)
        self.correct.assign_add(tf.reduce_sum(tf.cast(y_true == y_pred, tf.float32)))
        self.total.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        return self.correct / self.total

# Model with custom metrics
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[CustomAccuracy()])

model.fit(x_train, y_train, epochs=5, validation_split=0.1)


Epoch 1/5
1688/1688 [==============================] - 59s 35ms/step - loss: 0.1588 - custom_accuracy: 0.0984 - val_loss: 0.0439 - val_custom_accuracy: 0.0967
Epoch 2/5
1688/1688 [==============================] - 57s 34ms/step - loss: 0.0524 - custom_accuracy: 0.0990 - val_loss: 0.0350 - val_custom_accuracy: 0.0975
Epoch 3/5
1688/1688 [==============================] - 56s 33ms/step - loss: 0.0376 - custom_accuracy: 0.0990 - val_loss: 0.0340 - val_custom_accuracy: 0.0972
Epoch 4/5
1688/1688 [==============================] - 55s 32ms/step - loss: 0.0287 - custom_accuracy: 0.0989 - val_loss: 0.0340 - val_custom_accuracy: 0.0968
Epoch 5/5
1688/1688 [==============================] - 58s 34ms/step - loss: 0.0228 - custom_accuracy: 0.0988 - val_loss: 0.0311 - val_custom_accuracy: 0.0975


Custom Seed

In [ ]:
from tensorflow.keras import layers, models

# Set the custom seed
custom_seed = 42
np.random.seed(custom_seed)
tf.random.set_seed(custom_seed)

# Model
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_split=0.1)


Epoch 1/5
1688/1688 [==============================] - 58s 34ms/step - loss: 0.1523 - accuracy: 0.9524 - val_loss: 0.0511 - val_accuracy: 0.9857
Epoch 2/5
1688/1688 [==============================] - 57s 34ms/step - loss: 0.0539 - accuracy: 0.9834 - val_loss: 0.0428 - val_accuracy: 0.9870
Epoch 3/5
1688/1688 [==============================] - 55s 33ms/step - loss: 0.0375 - accuracy: 0.9882 - val_loss: 0.0294 - val_accuracy: 0.9915
Epoch 4/5
1688/1688 [==============================] - 56s 33ms/step - loss: 0.0276 - accuracy: 0.9912 - val_loss: 0.0339 - val_accuracy: 0.9907
Epoch 5/5
1688/1688 [==============================] - 57s 34ms/step - loss: 0.0222 - accuracy: 0.9926 - val_loss: 0.0320 - val_accuracy: 0.9907


Custom Call Back

In [ ]:
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('accuracy') > 0.95:
            print("\nReached 95% accuracy, stopping training.")
            self.model.stop_training = True

# Model with custom callback
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_split=0.1, callbacks=[CustomCallback()])


Custom Layer

In [ ]:
class CustomLayer(layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='zeros',
                                 trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

# Model with custom layer
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    CustomLayer(128),
    layers.Activation('relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_split=0.1)


Custom Model

In [ ]:
class CustomModel(models.Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.conv1 = layers.Conv2D(32, kernel_size=(3, 3), activation='relu')
        self.max_pool1 = layers.MaxPooling2D(pool_size=(2, 2))
        self.conv2 = layers.Conv2D(64, kernel_size=(3, 3), activation='relu')
        self.max_pool2 = layers.MaxPooling2D(pool_size=(2, 2))
        self.flatten = layers.Flatten()
        self.dense1 = layers.Dense(128, activation='relu')
        self.dropout = layers.Dropout(0.2)
        self.dense2 = layers.Dense(10, activation='softmax')

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = self.max_pool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dropout(x)
        return self.dense2(x)

# Model with custom model
model = CustomModel()

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_split=0.1)


Custom Training Loop

In [ ]:
# Model
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Custom training loop
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_fn(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)

for epoch in range(5):
    for step, (images, labels) in enumerate(train_dataset):
        loss = train_step(images, labels)
        if step % 100 == 0:
            print(f"Epoch {epoch + 1}, Step {step}, Loss {loss.numpy()}")


Custom Dropout

In [ ]:
class CustomDropout(layers.Layer):
    def __init__(self, rate=0.2, **kwargs):
        super(CustomDropout, self).__init__(**kwargs)
        self.rate = rate

    def call(self, inputs, training=None):
        if training:
            return tf.nn.dropout(inputs, rate=self.rate)
        return inputs

# Model with custom dropout
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    CustomDropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_split=0.1)


Custom Learning Rate Scheduler

In [ ]:
def custom_learning_rate_schedule(epoch, lr):
    if epoch % 10 == 0 and epoch > 0:
        return lr * 0.1
    return lr

# Model with custom learning rate scheduler
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

callback = tf.keras.callbacks.LearningRateScheduler(custom_learning_rate_schedule)

model.fit(x_train, y_train, epochs=5, validation_split=0.1, callbacks=[callback])


Epoch 1/5
1688/1688 [==============================] - 62s 36ms/step - loss: 0.1579 - accuracy: 0.9515 - val_loss: 0.0508 - val_accuracy: 0.9835 - lr: 0.0010
Epoch 2/5
1688/1688 [==============================] - 57s 34ms/step - loss: 0.0535 - accuracy: 0.9839 - val_loss: 0.0405 - val_accuracy: 0.9888 - lr: 0.0010
Epoch 3/5
1688/1688 [==============================] - 56s 33ms/step - loss: 0.0369 - accuracy: 0.9884 - val_loss: 0.0334 - val_accuracy: 0.9900 - lr: 0.0010
Epoch 4/5
1688/1688 [==============================] - 57s 34ms/step - loss: 0.0270 - accuracy: 0.9916 - val_loss: 0.0376 - val_accuracy: 0.9898 - lr: 0.0010
Epoch 5/5
1688/1688 [==============================] - 58s 34ms/step - loss: 0.0224 - accuracy: 0.9930 - val_loss: 0.0330 - val_accuracy: 0.9913 - lr: 0.0010
